In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.4/891.4 kB 15.2 MB/s eta 0:00:00


### 1. Data Split(Stratified Group K-Fold)

In [2]:
# %%bash

# mkdir /content/drive/MyDrive/like_lion/Computer_Vision/Object_Detection_Project/dataset
# mkdir /content/drive/MyDrive/like_lion/Computer_Vision/Object_Detection_Project/dataset/annotation
# cd /content/drive/MyDrive/like_lion/Computer_Vision/Object_Detection_Project/dataset
# kaggle datasets download sshikamaru/udacity-self-driving-car-dataset
# chmod 777 udacity-self-driving-car-dataset.zip
# unzip udacity-self-driving-car-dataset.zip
# mv data/export/_annotations.csv /content/drive/MyDrive/like_lion/Computer_Vision/Object_Detection_Project/dataset/annotation
# mv data/export .
# rm -rf data
# mv export data
# rm -rf udacity-self-driving-car-dataset.zip

In [3]:
# import os
# import shutil
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import StratifiedGroupKFold
# from tqdm import tqdm
# import yaml

# # Base path for dataset
# base_path = "/content/drive/MyDrive/like_lion/Computer_Vision/Object_Detection_Project/dataset/"
# image_base_path = base_path + "data/"  # 이미지 파일들이 저장된 디렉토리

# image_path_list = glob.glob(image_base_path+"*")

# image_file_name = []

# for image_path in image_path_list:
#   image_file_name.append(image_path.split('/')[9])

# # Load the annotations CSV
# annotation = pd.read_csv(base_path + "annotation/_annotations.csv")

# print(f"previous annotation len: {len(annotation)}")


# print(len(set(annotation['filename']) & set(image_file_name)))

# intersection_filenames = set(annotation['filename']).intersection(image_file_name)
# print(len(intersection_filenames))


# # 'filename'이 합집합에 포함된 행만 선택
# annotation = annotation[annotation['filename'].isin(intersection_filenames)]


# # print(annotation.columns)  # Index(['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax'], dtype='object')
# # 1. 'class' 컬럼 이름을 'class_name'으로 변경
# annotation.rename(columns={'class': 'class_name'}, inplace=True)

# # 2. 'class_name' 컬럼을 One-Hot Encoding하여 'class' 컬럼을 생성
# # One-Hot Encoding을 위한 컬럼 생성
# class_names = annotation['class_name'].unique()  # 고유한 클래스 이름을 추출
# class_mapping = {name: idx for idx, name in enumerate(class_names)}  # 클래스 이름과 인덱스 매핑

# # 'class_name'을 'class' 컬럼으로 변환 (One-Hot Encoding)
# annotation['class'] = annotation['class_name'].map(class_mapping)


# # Create StratifiedGroupKFold
# train_sgkf = StratifiedGroupKFold(n_splits=3)
# train_fold = train_sgkf.split(annotation, annotation['class'], groups=annotation['filename'])

# # Train, validation, and test splits
# train_fold = next(train_fold)
# train_annotation = annotation.iloc[train_fold[0]]
# tmp_annotation = annotation.iloc[train_fold[1]]

# test_sgkf = StratifiedGroupKFold(n_splits=2)
# test_fold = test_sgkf.split(tmp_annotation, tmp_annotation['class'], tmp_annotation['filename'])

# test_fold = next(test_fold)
# valid_annotation = tmp_annotation.iloc[test_fold[0]]
# test_annotation = tmp_annotation.iloc[test_fold[1]]


# train_annotation.drop_duplicates(['filename'], inplace = True)
# valid_annotation.drop_duplicates(['filename'], inplace = True)
# test_annotation.drop_duplicates(['filename'], inplace = True)

# # Define directory structure for YOLO
# yolo_base_path = "/content/drive/MyDrive/like_lion/Computer_Vision/Object_Detection_Project/yolov5_dataset/"
# image_dirs = ['train', 'valid', 'test']
# label_dirs = ['train', 'valid', 'test']

# # Create directories if they don't exist
# for image_dir, label_dir in zip(image_dirs, label_dirs):
#     os.makedirs(os.path.join(yolo_base_path, f"images/{image_dir}"), exist_ok=True)
#     os.makedirs(os.path.join(yolo_base_path, f"labels/{label_dir}"), exist_ok=True)


# # Function to convert annotations to YOLO format and move images
# def convert_to_yolo_format(annotation, image_set):
#     print(image_set)
#     # Paths for images and labels
#     image_dir = os.path.join(yolo_base_path, f"images/{image_set}")
#     label_dir = os.path.join(yolo_base_path, f"labels/{image_set}")

#     # Loop over each annotation
#     for _, row in tqdm(annotation.iterrows()):
#         # Image filename and paths
#         image_filename = row['filename']
#         image_path = os.path.join(image_base_path, image_filename)  # 이미지는 .data 폴더에서 찾기
#         label_filename = os.path.splitext(image_filename)[0] + '.txt'
#         label_path = os.path.join(label_dir, label_filename)

#         # Ensure the label directory exists before writing the file
#         os.makedirs(label_dir, exist_ok=True)  # Make sure the label directory exists

#         # Calculate YOLO format coordinates
#         img_width = row['width']
#         img_height = row['height']

#         # YOLO format: class_id center_x center_y width height (all values are relative to image size)
#         center_x = (row['xmin'] + row['xmax']) / 2 / img_width
#         center_y = (row['ymin'] + row['ymax']) / 2 / img_height
#         obj_width = (row['xmax'] - row['xmin']) / img_width
#         obj_height = (row['ymax'] - row['ymin']) / img_height

#         # Write to label file
#         with open(label_path, 'a') as label_file:
#             label_file.write(f"{row['class']} {center_x} {center_y} {obj_width} {obj_height}\n")

#         # Copy image to appropriate directory
#         shutil.copy(image_path, os.path.join(image_dir, image_filename))


# # Convert annotations for train, validation, and test datasets
# convert_to_yolo_format(train_annotation, 'train')
# convert_to_yolo_format(valid_annotation, 'valid')
# convert_to_yolo_format(test_annotation, 'test')

# print("YOLO dataset prepared.")


# # 3. data.yaml 파일 생성
# data_yaml = {
#     'train': '/content/drive/MyDrive/like_lion/Computer_Vision/Object_Detection_Project/yolov5_dataset/images/train',
#     'val': '/content/drive/MyDrive/like_lion/Computer_Vision/Object_Detection_Project/yolov5_dataset/images/valid',
#     'test': '/content/drive/MyDrive/like_lion/Computer_Vision/Object_Detection_Project/yolov5_dataset/images/test',
#     'nc': len(class_names),  # 클래스 개수 (One-Hot Encoding된 클래스의 개수)
#     'names': class_names.tolist()  # 클래스 이름 리스트
# }

# # data.yaml 경로
# yaml_path = '/content/drive/MyDrive/like_lion/Computer_Vision/Object_Detection_Project/yolov5_dataset/data.yaml'

# # YAML 파일로 저장
# with open(yaml_path, 'w') as f:
#     yaml.dump(data_yaml, f)

### 2. Model

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import gc
import os
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from tqdm import tqdm
import numpy as np
import torchvision.ops.boxes as box_ops
from PIL import Image
# from torchmetrics.detection.mean_ap import MeanAveragePrecision (commented out due to missing module)
# Install torchmetrics if needed: !pip install torchmetrics
from torchmetrics.detection.mean_ap import MeanAveragePrecision

# Clear cache once
torch.cuda.empty_cache()
gc.collect()

class ConvBNAct(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, act=True):
        super(ConvBNAct, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.act = nn.SiLU() if act else nn.Identity()

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))

class BottleNeck(nn.Module):
    def __init__(self, in_channels, out_channels, shortcut=True):
        super(BottleNeck, self).__init__()
        self.conv1 = ConvBNAct(in_channels, out_channels, 1, 1, 0)
        self.conv2 = ConvBNAct(out_channels, out_channels, 3, 1, 1)
        self.shortcut = shortcut and in_channels == out_channels

    def forward(self, x):
        out = self.conv2(self.conv1(x))
        return x + out if self.shortcut else out

class C3(nn.Module):
    def __init__(self, in_channels, out_channels, n=3):
        super(C3, self).__init__()
        self.conv1 = ConvBNAct(in_channels, out_channels // 2, 1, 1, 0)
        self.conv2 = ConvBNAct(in_channels, out_channels // 2, 1, 1, 0)
        self.conv3 = ConvBNAct(out_channels, out_channels, 1, 1, 0)
        self.bottlenecks = nn.Sequential(*[BottleNeck(out_channels // 2, out_channels // 2) for _ in range(n)])

    def forward(self, x):
        y1 = self.bottlenecks(self.conv1(x))
        y2 = self.conv2(x)
        return self.conv3(torch.cat((y1, y2), dim=1))

class SPPF(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(SPPF, self).__init__()
        self.conv1 = ConvBNAct(in_channels, out_channels, 1, 1, 0)
        self.conv2 = ConvBNAct(out_channels * 4, out_channels, 1, 1, 0)

    def forward(self, x):
        x = self.conv1(x)
        y1 = F.max_pool2d(x, 5, stride=1, padding=2)
        y2 = F.max_pool2d(x, 9, stride=1, padding=4)
        y3 = F.max_pool2d(x, 13, stride=1, padding=6)
        return self.conv2(torch.cat([x, y1, y2, y3], dim=1))

In [3]:
class YOLOv5(nn.Module):
    def initialize_anchors(self):
        # Anchors initialization for three scales (updated to include anchors)
        # Anchors for the three scales
        self.anchors = [
            [10, 13, 16, 30, 33, 23],  # Small scale anchors
            [30, 61, 62, 45, 59, 119],  # Medium scale anchors
            [116, 90, 156, 198, 373, 326]  # Large scale anchors
        ]
    def __init__(self, num_classes=11):

        self.initialize_anchors()  # Initialize anchors for detection layers
        super(YOLOv5, self).__init__()

        self.num_classes = num_classes

        # Backbone
        self.conv1 = ConvBNAct(3, 64, 6, 2, 2)
        self.conv2 = ConvBNAct(64, 128, 3, 2, 1)
        self.c3_1 = C3(128, 128, 3)
        self.conv3 = ConvBNAct(128, 256, 3, 2, 1)
        self.c3_2 = C3(256, 256, 6)
        self.conv4 = ConvBNAct(256, 512, 3, 2, 1)
        self.c3_3 = C3(512, 512, 9)
        self.conv5 = ConvBNAct(512, 1024, 3, 2, 1)
        self.sppf = SPPF(1024, 1024)

        # Neck
        self.c3_neck_1 = C3(1024 + 512, 512, 3)  # Adjusted to handle 1024+512=1536 channels
        self.c3_neck_2 = C3(512 + 256, 256, 3)

        # Upsample layers
        self.upsample = nn.Upsample(scale_factor=2, mode="nearest")

        # Prediction Layers (Head)
        self.detect_large = nn.Conv2d(1024, 3 * (self.num_classes + 5), 1)  # Large scale detection
        self.detect_medium = nn.Conv2d(512, 3 * (self.num_classes + 5), 1)  # Medium scale detection
        self.detect_small = nn.Conv2d(256, 3 * (self.num_classes + 5), 1)  # Small scale detection

    def forward(self, x):
        # Backbone
        x1 = self.conv1(x)
        x2 = self.conv2(x1)
        x2 = self.c3_1(x2)
        x3 = self.conv3(x2)
        x3 = self.c3_2(x3)
        x4 = self.conv4(x3)
        x4 = self.c3_3(x4)
        x5 = self.conv5(x4)
        x5 = self.sppf(x5)

        # Large scale detection
        detect_large = self.detect_large(x5)  # Output size: [batch_size, (num_classes + 5), h, w]

        # Neck
        x5_upsampled = self.upsample(x5)
        x4_concat = torch.cat((x5_upsampled, x4), dim=1)
        x4 = self.c3_neck_1(x4_concat)

        # Medium scale detection
        detect_medium = self.detect_medium(x4)

        x4_upsampled = self.upsample(x4)
        x3_concat = torch.cat((x4_upsampled, x3), dim=1)
        x3 = self.c3_neck_2(x3_concat)

        # Small scale detection
        detect_small = self.detect_small(x3)

        # 각 출력값을 반환할 때 바운드인포스, confidence, 클래스 분류
        predictions = [detect_small, detect_medium, detect_large]

        # 바운드 박스, confidence, 클래스 정보 분류
        pred_boxes = [pred[..., :4] for pred in predictions]  # [x_center, y_center, w, h]
        pred_conf = [torch.sigmoid(pred[..., 4:5]) for pred in predictions]  # confidence (sigmoid 적용)
        pred_cls = [torch.sigmoid(pred[..., 5:]) for pred in predictions]    # 클래스 (sigmoid 적용)

        return pred_boxes, pred_conf, pred_cls

### 3. Custom Dataset, DataLoader

In [3]:
# Dataset improvements and fixes
class ObjectDetectionDataset(Dataset):
    def __init__(self, image_dir, label_dir, transforms=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))]
        self.image_files = self.image_files[:len(self.image_files)//10]
        self.transforms = transforms

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Load image
        image_path = os.path.join(self.image_dir, self.image_files[idx])
        image = Image.open(image_path).convert("RGB")
        if self.transforms:
            image = self.transforms(image)

        # Load label
        label_path = os.path.join(self.label_dir, os.path.splitext(self.image_files[idx])[0] + '.txt')
        boxes = []
        labels = []
        with open(label_path, 'r') as f:
            for line in f.readlines():
                label, x_center, y_center, width, height = map(float, line.strip().split())
                labels.append(int(label))
                # YOLOv5 expects [x_center, y_center, width, height] normalized
                boxes.append([x_center, y_center, width, height])

        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.long)

        # YOLOv5 expects targets in the format: [class, x_center, y_center, width, height]
        targets = torch.cat((labels.unsqueeze(1).float(), boxes), dim=1)

        return image, targets

# Image transformations including normalization
image_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
def compute_loss(predictions, targets, anchors):
    # Computes the loss using anchor boxes for more accurate localization
    batch_size = targets.size(0)
    num_anchors = 3  # Assuming 3 anchors per detection layer
    targets = targets.view(batch_size, -1, 5)
    expanded_targets = targets.unsqueeze(1).expand(-1, num_anchors, -1, -1).contiguous().view(batch_size, -1, 5)
    pred_boxes, pred_conf, pred_cls = predictions
    anchor_tensors = [
        torch.tensor(anchor, device=targets.device).float().view(-1, 2) for anchor in anchors
    ]  # Create anchor tensors for each detection scale  # Create anchor tensors for each detection scale
    box_loss = sum(
        F.mse_loss(pred_box.view(batch_size, -1, 4), expanded_targets[..., 1:5]) for pred_box, anchor in zip(pred_boxes, anchor_tensors)
    )  # Calculate box loss using anchors
    conf_loss = sum(criterion(pred_conf[i].view(batch_size, -1, 1), expanded_targets[..., 0:1]) for i in range(len(pred_conf)))
    cls_loss = sum(criterion(pred_cls[i].view(batch_size, -1, num_classes), expanded_targets[..., 0:1].expand_as(pred_cls[i].view(batch_size, -1, num_classes))) for i in range(len(pred_cls)))
    return box_loss + conf_loss + cls_loss

In [ ]:
# Print summary
from torchsummary import summary


# Instantiate the model and print the summary with smaller input size to fit memory constraints
model = YOLOv5(num_classes=11).to('cuda' if torch.cuda.is_available() else 'cpu')

summary(model, (3, 512, 512))

### 4. Train

In [ ]:
# Hyperparameters
num_classes = 11
batch_size = 16
learning_rate = 0.001
epochs = 30

# Optimizer and loss function
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCEWithLogitsLoss()
mean_ap_metric = MeanAveragePrecision()

# DataLoader
train_loader = DataLoader(ObjectDetectionDataset("/content/drive/MyDrive/like_lion/Computer_Vision/Object_Detection_Project/yolov5_dataset/images/train", "/content/drive/MyDrive/like_lion/Computer_Vision/Object_Detection_Project/yolov5_dataset/labels/train", transforms=image_transforms),
                          batch_size=1, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(ObjectDetectionDataset("/content/drive/MyDrive/like_lion/Computer_Vision/Object_Detection_Project/yolov5_dataset/images/valid", "/content/drive/MyDrive/like_lion/Computer_Vision/Object_Detection_Project/yolov5_dataset/labels/valid", transforms=image_transforms),
                        batch_size=1, shuffle=False, num_workers=4, pin_memory=True)

In [ ]:
# Training loop
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    for images, targets in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}"):
        images, targets = images.to(device), targets.to(device)

        print(images[0].shape) # torch.Size([3, 512, 512])
        print(targets[0].shape) # torch.Size([1, 5]) # class, x_center, y_center, width, height

        optimizer.zero_grad()
        predictions = model(images)
        print(len(predictions)) # 3


        print(len(predictions[0])) # 3
        print(predictions[0])
        print(len(predictions[1])) # 3
        print(len(predictions[2])) # 3

        loss = compute_loss(predictions, targets, model.anchors)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    print(f"Epoch {epoch + 1}, Training Loss: {train_loss / len(train_loader)}")

    # Validation and mAP calculation
    model.eval()
    mean_ap_metric.reset()
    with torch.no_grad():
        for images, targets in val_loader:
            images, targets = images.to(device), targets.to(device)
            pred_boxes, pred_conf, pred_cls = model(images)
            detections = []
            for i in range(len(pred_boxes)):
                for j in range(len(pred_boxes[i])):
                    detection = torch.cat((pred_boxes[i][j], pred_conf[i][j], pred_cls[i][j]), dim=-1)
                    detections.append(detection)
            mean_ap_metric.update(detections, targets)

    mAP = mean_ap_metric.compute()
    print(f"Epoch {epoch + 1}, Validation mAP: {mAP}")

In [ ]:
# Training loop
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    for images, targets in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}"):
        images, targets = images.to(device), targets.to(device)

        print(images[0].shape) # torch.Size([3, 512, 512])
        print(targets[0].shape) # torch.Size([1, 5]) # class, x_center, y_center, width, height

        optimizer.zero_grad()
        predictions = model(images)
        print(len(predictions)) # 3


        print(len(predictions[0])) # 3
        print(predictions[0])
        print(len(predictions[1])) # 3
        print(len(predictions[2])) # 3

        loss = compute_loss(predictions, targets, model.anchors)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    print(f"Epoch {epoch + 1}, Training Loss: {train_loss / len(train_loader)}")

    # Validation and mAP calculation
    model.eval()
    mean_ap_metric.reset()
    with torch.no_grad():
        for images, targets in val_loader:
            images, targets = images.to(device), targets.to(device)
            pred_boxes, pred_conf, pred_cls = model(images)
            detections = []
            for i in range(len(pred_boxes)):
                for j in range(len(pred_boxes[i])):
                    detection = torch.cat((pred_boxes[i][j], pred_conf[i][j], pred_cls[i][j]), dim=-1)
                    detections.append(detection)
            mean_ap_metric.update(detections, targets)

    mAP = mean_ap_metric.compute()
    print(f"Epoch {epoch + 1}, Validation mAP: {mAP}")